In [ ]:
%load_ext autoreload
%autoreload 2

# resample data uing MNE
import os, sys

code_dir = os.path.expandvars('$OSCBAGDIS_DATAPROC_CODE')
sys.path.append(code_dir)

import mne
from mne.preprocessing import ICA
import multiprocessing as mpr
import numpy as np
import utils
import json
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import utils_preproc as upre
import matplotlib as mpl
import globvars as gv


In [ ]:
# TODO: look only at files where there are several tremor starts

In [ ]:
# subjinds = [8,9,10]
# tasks = ['hold' , 'move', 'rest']
# medstates = ['on','off']


# for subjind in subjinds:
#     sis = '{:02d}'.format(subjind)
#     for medstate in medstates:
#         for task in tasks:

#             #fnames_noext = ['S{}_off_{}'.format(sis,task), 'S{}_on_{}'.format(sis,task)]
#             #fnames_noext = ['S01_off_hold', 'S01_on_hold']
#             #fnames_noext = ['S01_off_move', 'S01_on_move']
#             #fnames_noext = ['S02_off_move', 'S02_on_move']
#             rawname_ = 'S{}_{}_{}'.format(sis,medstate,task)


# rawname_ = 'S04_off_move'

In [ ]:
#gc.collect()

%run -i run_genfeats.py -r S02_off_hold --bands fine #--plot_only #--plot_types=feat_timecourse

In [ ]:
# S04 right hand tremor type -- smooth decay

In [ ]:
rawind = 0

In [ ]:
raws_lfp_perside

In [ ]:
raws_emg_rectconv

In [ ]:
rectconv = raws_emg_rectconv[rawind]

In [ ]:
rectconv.set_annotations(anns_pri[rawind])

In [ ]:
emg_perside = utils.getEMGperHand(rectconv)

In [ ]:
emg_perside

In [ ]:
maintremside

In [ ]:
opside = utils.getOppositeSideStr(maintremside)
ops_letter = opside[0].upper()

In [ ]:
print(mts_letter)

In [ ]:
anns = anns_pri[rawind]
anninds_notrem = np.where( anns.description == 'notrem_{}'.format(mts_letter) )[0]
ind_longest = None
dur_longest = 0
for anni,ann in enumerate(anns):
    if ann['duration'] > dur_longest:
        dur_longest = ann['duration']
        ind_longest = anni
ann_bl = anns[ind_longest]
print(ind_longest, ann_bl)

In [ ]:
print(anns.description)

In [ ]:
#gv.gen_subj_info['S04']['move_side']

In [ ]:
int_types_L = ['trem_L', 'notrem_L', 'hold_L', 'move_L', 'undef_L', 'holdtrem_L', 'movetrem_L']
int_types_R = ['trem_R', 'notrem_R', 'hold_R', 'move_R', 'undef_R', 'holdtrem_R', 'movetrem_R']
class_ids = {}
for ind, it in enumerate(int_types_L):
    class_ids[it] = ind+1
for ind, it in enumerate(int_types_R):
    class_ids[it] = -ind-1

int_types_basic = ['trem', 'notrem', 'hold', 'move']


In [ ]:
sides_hand = [mts_letter]


In [ ]:
class_ids

In [ ]:
# lofreq
# raws_lfponly[rawind].set_annotations( raws_lfponly[rawind].annotations + anns_artif)
# lfp_perside_cur = utils.getLFPperSide(raws_lfponly[rawind])

In [ ]:
raws_lfp_highres[rawind].set_annotations( raws_lfponly[rawind].annotations + anns_artif)
lfp_perside_cur = utils.getLFPperSide(raws_lfp_highres[rawind])

In [ ]:
raws_srconly[rawind].set_annotations( raws_lfponly[rawind].annotations + anns_artif)
src_perside_cur = utils.getRawPerSide(raws_srconly[rawind])

In [ ]:
#gv.gen_subj_info[subj]['lfpchan_used_in_paper']

In [ ]:
lfp_maintrem_side = lfp_perside_cur[ops_letter]
src_maintrem_side = src_perside_cur[ops_letter]
emg_maintrem_side = emg_perside[maintremside]

lfp_maintrem_side.pick_channels([mainLFPchan])

src_maintrem_side.set_annotations( utils.removeAnnsByDescr(src_maintrem_side.annotations, ['BAD_LFP'])  )

In [ ]:
#lfp_maintrem_side.ch_names

In [ ]:
emg_maintrem_side.plot();

In [ ]:
print( lfp_maintrem_side.annotations, emg_maintrem_side.annotations )

print(maintremside)

In [ ]:
# evt_dict = { 'trem_{}'.format(mts_letter), 'notrem_{}'.format(mts_letter),
#            'move_{}'.format(mts_letter), 'hold_{}'.format(mts_letter):}

In [ ]:
#help(mne.events_from_annotations)

In [ ]:
#regexp='^(?![Bb][Aa][Dd]|[Ee][Dd][Gg][Ee]).*$'
# get onsets
# onsets_mts, lbl = mne.events_from_annotations(emg_perside[opside], event_id=class_ids, 
#                             use_rounding=True, chunk_duration=None, verbose=None)

onsets_mts, lbl = mne.events_from_annotations(lfp_maintrem_side, 
                                              event_id=class_ids, 
                            use_rounding=True, chunk_duration=None, verbose=None)

In [ ]:
anns_tmp = copy.deepcopy(lfp_maintrem_side.annotations)
onset_tmp = anns_tmp.onset 
raw_tmp = lfp_maintrem_side.copy()
raw_tmp.set_annotations( mne.Annotations(anns_tmp.onset+anns_tmp.duration, 
                                        len(anns_tmp) * [0.1], anns_tmp.description))

outsets_mts, lbl = mne.events_from_annotations(raw_tmp, 
                                              event_id=class_ids, 
                            use_rounding=True, chunk_duration=None, verbose=None)

In [ ]:
onsets_mts, onsets_mts[:,0]/256

In [ ]:
revdict = {}
import copy
classs_ids_flt = copy.deepcopy(class_ids)
# set class label for all basic interval types
for itb in class_ids:
    cid = class_ids[itb]
    if cid in onsets_mts[:,2]:
        revdict[cid] = itb
    else:
        del classs_ids_flt[itb]
        

In [ ]:
classs_ids_flt

In [ ]:
# import copy
# class_ids_restr = copy.deepcopy(class_ids)
# for i in range(len(onsets_mts)):
#     if onsets_mts[i][2] not in class_ids_restr.values():
        

In [ ]:
#src_maintrem_side.annotations.description

In [ ]:
bl = ann_bl['onset'], ann_bl['onset'] + ann_bl['duration']
bl = None
epochs_emg = mne.Epochs(emg_maintrem_side, events=onsets_mts, tmax=3, tmin = -2,
                    event_id=classs_ids_flt, baseline = bl)

epochs_lfp = mne.Epochs(lfp_maintrem_side, events=onsets_mts, tmax=3, tmin = -2,
                    event_id=classs_ids_flt, baseline = bl)

epochs_src = mne.Epochs(src_maintrem_side, events=onsets_mts, tmax=3, tmin = -2,
                    event_id=classs_ids_flt, baseline = bl)


epochs_mid_emg = mne.Epochs(emg_maintrem_side, events=onsets_mts, tmax=4, tmin = 1,
                    event_id=classs_ids_flt, baseline = bl)

epochs_mid_lfp = mne.Epochs(lfp_maintrem_side, events=onsets_mts, tmax=4, tmin = 1,
                    event_id=classs_ids_flt, baseline = bl)

epochs_mid_src = mne.Epochs(src_maintrem_side, events=onsets_mts, tmax=4, tmin = 1,
                    event_id=classs_ids_flt, baseline = bl)


In [ ]:
epochs_end_emg = mne.Epochs(emg_maintrem_side, events=outsets_mts, tmin=-3, tmax = 1,
                    event_id=classs_ids_flt, baseline = bl)

epochs_end_lfp = mne.Epochs(lfp_maintrem_side, events=outsets_mts, tmin=-3, tmax = 1,
                    event_id=classs_ids_flt, baseline = bl)

epochs_end_src = mne.Epochs(src_maintrem_side, events=outsets_mts, tmin=-3, tmax = 1,
                    event_id=classs_ids_flt, baseline = bl)


epochs_endmid_emg = mne.Epochs(emg_maintrem_side, events=outsets_mts, tmin=-4, tmax = -1,
                    event_id=classs_ids_flt, baseline = bl)

epochs_endmid_lfp = mne.Epochs(lfp_maintrem_side, events=outsets_mts, tmin=-4, tmax = -1,
                    event_id=classs_ids_flt, baseline = bl)

epochs_endmid_src = mne.Epochs(src_maintrem_side, events=outsets_mts, tmin=-4, tmax = -1,
                    event_id=classs_ids_flt, baseline = bl)

In [ ]:
#%matplotlib qt
#epochs_emg[int_type].plot();

In [ ]:
fbands

In [ ]:
%matplotlib inline

In [ ]:
int_type = 'trem_{}'.format(mts_letter)

# LFP start

In [ ]:
epochs_lfp_per_band = {}
for bandname in fband_names_fine_inc_HFO:
    lf,hf = fbands[bandname]
    epochs_lfp_cb = epochs_lfp.copy()
    epochs_lfp_cb.load_data();
    epochs_lfp_cb.filter(l_freq=lf,h_freq=hf, verbose=False)
    epochs_lfp_per_band[bandname] = epochs_lfp_cb.copy()
    
    epochs_lfp_cb.apply_hilbert(envelope=True)
    
    #fig = plt.figure(figsize=(15,4))
    print(bandname)
    fig = epochs_lfp_cb[int_type].plot_image(combine='mean', show=0)#,fig=fig)
    ax = fig[0].axes[0]
    ax.set_title(ax.title._text + '  ' + bandname)
    #title.suptitle(bandname)

# LFP end

In [ ]:
epochs_lfp_per_band = {}
for bandname in fband_names_fine:
    lf,hf = fbands[bandname]
    epochs_lfp_cb = epochs_end_lfp.copy()
    epochs_lfp_cb.load_data();
    epochs_lfp_cb.filter(l_freq=lf,h_freq=hf, verbose=False)
    epochs_lfp_per_band[bandname] = epochs_lfp_cb.copy()
    
    epochs_lfp_cb.apply_hilbert(envelope=True)
    
    #fig = plt.figure(figsize=(15,4))
    print(bandname)
    fig = epochs_lfp_cb[int_type].plot_image(combine='mean', show=0)#,fig=fig)
    ax = fig[0].axes[0]
    ax.set_title(ax.title._text + '  ' + bandname)
    #title.suptitle(bandname)

# SRC start

In [ ]:
epochs_emg_cb[int_type]

In [ ]:
epochs_src[int_type]

In [ ]:
int_type

In [ ]:
#src_maintrem_side.annotations

In [ ]:
epochs_src_per_band = {}
for bandname in fband_names_fine:
    lf,hf = fbands[bandname]
    epochs_src_cb = epochs_src.copy()
    #display(epochs_src_cb[int_type])
    epochs_src_cb.load_data();
    epochs_src_cb.filter(l_freq=lf,h_freq=hf, verbose=False)
    epochs_src_per_band[bandname] = epochs_src_cb.copy()
    
    epochs_src_cb.apply_hilbert(envelope=True)
    
    #fig = plt.figure(figsize=(15,4))
    print(bandname)
    fig = epochs_src_cb[int_type].plot_image(combine='mean', show=0)#,fig=fig)
    ax = fig[0].axes[0]
    ax.set_title(ax.title._text + '  ' + bandname)
    #title.suptitle(bandname)

In [ ]:
#epochs_emg[int_type]

In [ ]:
epochs_emg[int_type].plot_image(combine='mean') #,ts_args=dict(vlines=[0, rts.mean() / 1000.]))
#epochs_end_emg[int_type].plot_image(combine='mean') #,ts_args=dict(vlines=[0, rts.mean() / 1000.]))

In [ ]:
%matplotlib inline
int_type = 'trem_{}'.format(mts_letter)

epochs_emg[int_type].plot_image(combine='mean') #,ts_args=dict(vlines=[0, rts.mean() / 1000.]))
epochs_lfp[int_type].plot_image(combine='mean') #,ts_args=dict(vlines=[0, rts.mean() / 1000.]))
epochs_mid_lfp[int_type].plot_psd(fmax=60);

In [ ]:
int_type = 'notrem_{}'.format(mts_letter)
epochs_mid_emg[int_type].plot_image(combine='mean')
epochs_mid_lfp[int_type].plot_image(combine='mean')
epochs_mid_lfp[int_type].plot_psd(fmax=60);